# Simulation Results

This notebook summarizes the results of the optimization experiments.

In [1]:
import json
import os

import numpy as np
import pandas as pd

In [2]:
def load_meta(**params):
    """Return metafile based on simulation properties."""
    metafn = os.path.join(params['base_dirn'],
                          params['metafn_fmt'].format(**params))
    return json.load(open(metafn, 'r'))


def sim_results(**params):
    """Summarize simulation results for an algo-func combination."""
    meta = load_meta(**params)
    expxkmin, fxk = meta['exp_fxkmin'], np.array(meta['f(xk)'])
    delta_fxk = expxkmin - fxk[~np.isnan(fxk)]
    nsteps = np.array(meta['nsteps'])
    fxk_thresh = params.get('fxk_thresh', 1.)
    nfoundmin = sum(np.fabs(delta_fxk) < fxk_thresh)
    elapsed_sec = np.array(meta['elapsed_sec'])
    return dict(
        alg=meta['alg'],
        func=meta['func'],
        ntrials=meta['ntrials'],
        nfoundmin=nfoundmin,
        delta_fxk_min=np.min(delta_fxk),
        delta_fxk_max=np.max(delta_fxk),
        delta_fxk_median=np.median(delta_fxk),
        nsteps_min=np.min(nsteps),
        nsteps_max=np.max(nsteps),
        nsteps_median=np.median(nsteps),
        elapsed_sec_median=np.median(elapsed_sec),
        elapsed_sec_mean=np.mean(elapsed_sec),
    )

In [3]:
opts = {
    'base_dirn': './sims/',
    'savefn_fmt': '{alg}-{func}-steps-{trial:02d}.npy',
    'metafn_fmt': '{alg}-{func}-meta.json',
}

params = [
    {
        'alg': 'gradient_descent',
        'func': 'rosenbrock',
    },
    {
        'alg': 'gradient_descent',
        'func': 'goldstein_price',
        'fxk_thresh': 10,
    },
    {
        'alg': 'bfgs',
        'func': 'rosenbrock',
    },
    {
        'alg': 'bfgs',
        'func': 'goldstein_price',
        'fxk_thresh': 10,
    },
    {
        'alg': 'simulated_annealing',
        'func': 'rosenbrock',
    },
    {
        'alg': 'simulated_annealing',
        'func': 'goldstein_price',
        'fxk_thresh': 10,
    },
    {
        'alg': 'simulated_annealing',
        'func': 'bartels_conn',
    },
    {
        'alg': 'simulated_annealing',
        'func': 'egg_crate',
    },
    {
        'alg': 'particle_swarm',
        'func': 'rosenbrock',
    },
    {
        'alg': 'particle_swarm',
        'func': 'goldstein_price',
        'fxk_thresh': 10,
    },
    {
        'alg': 'particle_swarm',
        'func': 'bartels_conn',
    },
    {
        'alg': 'particle_swarm',
        'func': 'egg_crate',
    },
]

results_df = pd.DataFrame()
for param in params:
    param.update(**opts)
    results = sim_results(**param)
    results_df = results_df.append(results, ignore_index=True)

results_df = results_df.astype({'ntrials':int,
                                'nfoundmin':int,
                                'nsteps_median':int})

disp_cols = [
    'alg',
    'func',
    'ntrials',
    'nfoundmin',
    'delta_fxk_median',
    'nsteps_median',
]

results_df[disp_cols].sort_values(by=['alg']).style.hide_index()

alg,func,ntrials,nfoundmin,delta_fxk_median,nsteps_median
bfgs,rosenbrock,12,12,-0.000000,28
bfgs,goldstein_price,12,6,-13.500000,17
gradient_descent,rosenbrock,12,12,-0.000123,8320
gradient_descent,goldstein_price,12,3,-27.000000,2139
particle_swarm,rosenbrock,12,11,-0.193986,200
particle_swarm,goldstein_price,12,10,-0.376091,2000
particle_swarm,bartels_conn,12,11,-0.088850,300
particle_swarm,egg_crate,12,12,-0.248673,2000
simulated_annealing,rosenbrock,12,12,-0.176289,200
simulated_annealing,goldstein_price,12,6,-44.026184,2000
